In [1]:
!pip install git+https://github.com/akanametov/deepl.git#egg=deepl

  Cloning https://github.com/akanametov/deepl.git to /tmp/pip-install-kvu8q2w6/deepl
  Running command git clone -q https://github.com/akanametov/deepl.git /tmp/pip-install-kvu8q2w6/deepl
  Created wheel for deepl: filename=deepl-0.1-cp37-none-any.whl size=11430 sha256=8f5c890c6e60695ba25e7fef32591e823ce364a8475a5860a417808a57c46722
  Stored in directory: /tmp/pip-ephem-wheel-cache-ootuamdi/wheels/08/fe/70/dfa6150a653ff950af33a7358c093de4e8af97b42192bd6420
Successfully built deepl


In [2]:
!wget https://pjreddie.com/media/files/mnist_train.csv

--2021-06-10 09:04:17--  https://pjreddie.com/media/files/mnist_train.csv
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109575994 (104M) [application/octet-stream]
Saving to: ‘mnist_train.csv’

mnist_train.csv     100%[===================>] 104.50M  31.1MB/s    in 3.7s    

2021-06-10 09:04:21 (28.0 MB/s) - ‘mnist_train.csv’ saved [109575994/109575994]



In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

train = pd.read_csv('sample_data/mnist_train_small.csv', header=None).sample(1000)
assert len(np.unique(train.iloc[:, 0])) == 10

x = train.drop(labels=0, axis=1).to_numpy()
y = train[0].values.copy()

In [4]:
from deepl import nn
from deepl.nn import functional as F
from deepl.criterion import CrossEntropyLoss
from deepl.metrics import accuracy
from deepl.optimizer import Adam
from deepl.data import Dataset, DataLoader

# define network architecture
class CNN(nn.Network):
    def __init__(self,):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=(3,3), stride=(2,2))
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(4, 6, kernel_size=(3,3), stride=(2,2))
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(6, 8, kernel_size=(3,3), stride=(2,2))
        self.relu3 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(8*2*2, 16)
        self.relu4 = nn.ReLU()
        self.fc = nn.Linear(16, 10)
        
    def forward(self, x): # define forward path
        
        fx = self.conv1(x)
        fx = self.relu1(fx)
        fx = self.conv2(fx)
        fx = self.relu2(fx)
        fx = self.conv3(fx)
        fx = self.relu3(fx)
        
        fx = self.flatten(fx)
        
        fx = self.linear1(fx)
        fx = self.relu4(fx)
        fx = self.fc(fx)
        
        return fx

# define your dataset
class MyDataset(Dataset):
    def __init__(self, x, y, mode='train'):
        super().__init__()
        self.x = x
        self.y = y
        self.mode=mode
        
    def __len__(self,):
        return len(self.x)
    
    def __getitem__(self, idx):
        #print(idx)
        xi = self.x[idx]#.iloc[idx].values
        #print(xi.shape)
        xi = xi.reshape(len(xi), 1, 28, 28)/255
        if self.mode=='test':
            return xi
        yi = self.y[idx].astype(int)
        return xi, yi

In [5]:
dataset = MyDataset(x, y)
dataloader = DataLoader(dataset, batch_size=50)

# assign model, critreion and optimizer
model = CNN()
criterion = CrossEntropyLoss()
optimizer = Adam(lr=0.0005)

In [6]:
epochs=100
history={'loss':[], 'acc':[]}

# Train loop
for epoch in range(epochs):
    e_loss, e_acc = 0., 0.
    for (x, y) in dataloader:
        p = model(x)#.flatten()                   # obtain prediction
        loss = criterion(p, y)                   # calculate loss 
        acc = accuracy(np.argmax(p,axis=1), y) # calculate accuracy
        grad = criterion.backward()              # get gradient of loss function 
        model.backward(grad)                     # backward propogate of gradient through network
        params = optimizer.step(model.parameters(), model.grads()) # calculate new parameters by optimizer
        model.update(params)                     # update parameters of network
        
        e_loss += loss.item()/len(dataloader)
        e_acc += acc.item()/len(dataloader)
    print(f'Epoch {epoch+1}/{epochs} | loss: {e_loss:.4f} | acc: {e_acc:.4f}')
    history['loss'].append(e_loss)               # save loss value
    history['acc'].append(e_acc)                 # save accuracy value

Epoch 1/100 | loss: 2.3210 | acc: 0.1210
Epoch 2/100 | loss: 2.2941 | acc: 0.1850
Epoch 3/100 | loss: 2.2794 | acc: 0.1820
Epoch 4/100 | loss: 2.2638 | acc: 0.1890
Epoch 5/100 | loss: 2.2400 | acc: 0.2120
Epoch 6/100 | loss: 2.2028 | acc: 0.2340
Epoch 7/100 | loss: 2.1405 | acc: 0.3040
Epoch 8/100 | loss: 2.0429 | acc: 0.3500
Epoch 9/100 | loss: 1.9130 | acc: 0.3650
Epoch 10/100 | loss: 1.7878 | acc: 0.3900
Epoch 11/100 | loss: 1.6743 | acc: 0.4460
Epoch 12/100 | loss: 1.5689 | acc: 0.4950
Epoch 13/100 | loss: 1.4818 | acc: 0.5280
Epoch 14/100 | loss: 1.4093 | acc: 0.5510
Epoch 15/100 | loss: 1.3508 | acc: 0.5840
Epoch 16/100 | loss: 1.3023 | acc: 0.6000
Epoch 17/100 | loss: 1.2610 | acc: 0.6160
Epoch 18/100 | loss: 1.2248 | acc: 0.6230
Epoch 19/100 | loss: 1.1927 | acc: 0.6360
Epoch 20/100 | loss: 1.1638 | acc: 0.6370
Epoch 21/100 | loss: 1.1374 | acc: 0.6430
Epoch 22/100 | loss: 1.1125 | acc: 0.6530
Epoch 23/100 | loss: 1.0895 | acc: 0.6590
Epoch 24/100 | loss: 1.0681 | acc: 0.6650
E